# Day 24, blowing in the wind

Path search with a twist(er)! We need to encode the blizzards; they make a full rotation every $lcm(a, b) = \frac{\lvert a \cdot b \rvert} {gcd(a, b)}$ steps, where $a = x - 2, b = y - 2$ to ignore the borders. That's a small enough value to just generate up front and store as a data structure that lets you test if at a given time, a given position on the map is a blizzard.

Then, when path finding, we need to treat the time as cyclical too; a state at position $(x, y)$ and time $t$ is the same state as $(x, y)$ at time $t + cycle$; the exact same steps are available. Essentially, the map is 3-dimensional, it has width, height, and time!

In [1]:
# pyright: strict
import math
from collections import deque
from dataclasses import dataclass
from typing import Final, Iterator, Literal, NamedTuple, Self, assert_never


def lcm(a: int, b: int) -> int:
    return abs(a * b) // math.gcd(a, b)


class Pos(NamedTuple):
    x: int = 0
    y: int = 0

    def __add__(self, other: Self) -> Self:
        if not isinstance(other, Pos):
            return NotImplemented
        return Pos(self.x + other.x, self.y + other.y)


OPTIONS: Final[tuple[Pos, ...]] = (
    Pos(0, 0),  # wait in place
    Pos(0, -1),
    Pos(1, 0),
    Pos(0, 1),
    Pos(-1, 0),
)


class Blizzards:
    times: tuple[tuple[tuple[bool, ...], ...]]
    length: int
    shape: tuple[int, int]

    def __init__(self, map: str) -> None:
        rows = map.splitlines()
        self.shape = h, w = len(rows), len(rows[0])
        # build 2 deques for each line and each column, one per direction
        ups: list[deque[bool]]
        downs: list[deque[bool]]
        lefts: list[deque[bool]]
        rights: list[deque[bool]]
        ups, downs, lefts, rights = (
            [deque() for _ in range(w - 2)],
            [deque() for _ in range(w - 2)],
            [deque() for _ in range(h - 2)],
            [deque() for _ in range(h - 2)],
        )
        for row, left, right in zip(rows[1:-1], lefts, rights):
            for char, up, down in zip(row[1:-1], ups, downs):
                l = r = u = d = False
                match char:
                    case "<":
                        l = True
                    case ">":
                        r = True
                    case "^":
                        u = True
                    case "v":
                        d = True
                    case _:
                        pass
                left.append(l)
                right.append(r)
                up.append(u)
                down.append(d)
        self.length = cycle = lcm(h - 2, w - 2)
        # build sets of occupied positions, one per cycle
        times: list[tuple[tuple[bool, ...], ...]] = []
        for _ in range(cycle):
            time: list[tuple[bool, ...]] = []
            upsdowns = [(iter(up), iter(down)) for up, down in zip(ups, downs)]
            for left, right in zip(lefts, rights):
                line: list[bool] = [any((l, r, next(u), next(d))) for l, r, (u, d) in zip(left, right, upsdowns)]
                time.append(tuple(line))
                left.rotate(-1)
                right.rotate(1)
            times.append(tuple(time))
            for up, down in zip(ups, downs):
                up.rotate(-1)
                down.rotate(1)
        self.times = tuple(times)

    def __contains__(self, tpos: tuple[int, Pos]) -> bool:
        t, (x, y) = tpos
        if not (0 < x < self.shape[1] - 1 and 0 < y < self.shape[0] - 1):
            return False
        return self.times[t % self.length][y - 1][x - 1]


@dataclass(frozen=True, slots=True)
class ValleyStep:
    pos: Pos = Pos()
    steps: int = 0

    def traverse(self, valley: "BlizzardValley") -> Iterator[Self]:
        pos, steps = self.pos, self.steps + 1
        for delta in OPTIONS:
            new_pos = pos + delta
            if (steps, new_pos) not in valley:
                continue
            yield self.__class__(new_pos, steps)


class BlizzardValley:
    blizzards: Blizzards
    shape: tuple[int, int]
    start: Pos
    end: Pos

    def __init__(self, map: str) -> None:
        blizzards = self.blizzards = Blizzards(map)
        self.shape = h, w = blizzards.shape
        self.start = Pos(map.index("."), 0)
        self.end = Pos(map.rindex(".") % (w + 1), h - 1)

    def __contains__(self, tpos: tuple[int, Pos]) -> bool:
        t, (x, y) = tpos
        h, w = self.shape
        # within the bounds, but left and right walls are impassable
        if not (0 <= y < h and 0 < x < w - 1):
            return False
        if y == 0:
            return x == self.start.x
        if y == h - 1:
            return x == self.end.x
        return tpos not in self.blizzards

    def find_path(self, reverse: bool = False, time: int = 0) -> int:
        start = ValleyStep(self.end if reverse else self.start, time)
        goal = self.start if reverse else self.end
        cycle = self.blizzards.length
        queue = deque([start])
        seen: dict[tuple[int, Pos], int] = {(start.steps % cycle, start.pos): start.steps}
        while queue:
            step = queue.popleft()
            if step.pos == goal:
                return step.steps
            for next in step.traverse(self):
                key = (next.steps % cycle, next.pos)
                if seen.get(key, math.inf) <= next.steps:
                    continue
                seen[key] = next.steps
                queue.append(next)
        raise ValueError("no path found")
    

example = BlizzardValley(
    """\
#.######
#>>.<^<#
#.<..<<#
#>v.><>#
#<^v^^>#
######.#
"""
)

example_steps = example.find_path()
assert example_steps == 18

In [2]:
import aocd


valley = BlizzardValley(aocd.get_data(day=24, year=2022))
steps = valley.find_path()
print("Part 1:", steps)

Part 1: 292


## Part 2, the Lord of the Snacks

We are going there and back again. Yes, someone left their snacks and is willing to brave the blizzards an additional 2 times!

Just re-run the simulation in reverse, at a new starting time. Good thing I pre-calculated all those states!

In [3]:
example_return = example.find_path(reverse=True, time=example_steps)
assert example.find_path(time=example_return) == 54

In [4]:
print("Part 2:", valley.find_path(time=valley.find_path(reverse=True, time=steps)))

Part 2: 816
